In [0]:
%pip install databricks-vectorsearch
%restart_python

In [0]:
from mlflow.deployments import get_deploy_client
from pyspark.sql.functions import pandas_udf
import pandas as pd

client = get_deploy_client("databricks")
EMBEDDING_ENDPOINT_NAME = "databricks-bge-large-en"


1. Generate a Query Embedding

In [0]:
# Example: Get embedding for a query
query_text = "Explain reinforcement learning in simple terms"
query_embedding = client.predict(endpoint=EMBEDDING_ENDPOINT_NAME, inputs={"input": [query_text]})["data"][0]["embedding"]


2. Search the Vector Index

In [0]:
df = spark.table("genai.default.pdf_chunks_embedded")
df.selectExpr("size(embedding)").distinct().show()

In [0]:

from databricks.vector_search.client import VectorSearchClient
import pandas as pd

# Step 1: Initialize vector search client
vsc = VectorSearchClient(disable_notice=True)

# Step 2: Define index
index = vsc.get_index(
    endpoint_name="genai_vec_search_shankho",  # your serving endpoint
    index_name="genai.default.vec_index"       # catalog.schema.index_name
)

# Step 3: Embed your query
query = "How many reading questions are there in per section"
query_embedding = client.predict(endpoint=EMBEDDING_ENDPOINT_NAME, inputs={"input": [query]})["data"][0]["embedding"]

# Step 4: Perform search
results_df: pd.DataFrame = index.similarity_search(
    query_vector=query_embedding,
    columns=["chunk_id", "content"],
    num_results=5
)

display(results_df)
